# Attach Background Image

## Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import warnings
import json

from IPython.display import clear_output
from PIL import Image
import matplotlib.pyplot as plt

warnings.filterwarnings(action='ignore')

%matplotlib inline

## Dataset

In [ ]:
train_set = pd.read_csv('dataset/dot_train.csv')
test_set = pd.read_csv('dataset/dot_test.csv')

In [ ]:
with open('dataset/data_container.json') as f:
    img_matching = json.load(f)
    
dot = img_matching['DOT']

## Backgroud Image

In [ ]:
size = 512
bg = np.full((size, size, 3), 0)
bg = bg.astype(np.uint8)
bg_img = Image.fromarray(bg)

In [ ]:
plt.imshow(bg_img);

In [ ]:
bh, bw = bg_img.size

## Attach Sub Pattern Images and Background Image

In [ ]:
def group_sub_pattern(sem_no_list: pd.Series, dir_: str) -> dict:
    container = {}
    for i in sem_no_list:
        sem_no = f'{i:05d}'
        try: pattern_list = os.listdir(f'{dir_}/{sem_no}')
        except: continue
        container[sem_no] = pattern_list
    return container

In [ ]:
def attach_bg(container: dict, dir_: str, dataset_name: str, output_path: str, extension: str='png') -> None:
    total_subpattern = len(container.keys())
    
    for n, (i, v) in enumerate(container.items()):
        print(f'[{n + 1}/{total_subpattern}] Processing {i} ...')
        dir_name = f'{dir_}/{i}'
        try: filelist = os.listdir(dir_name)
        except: continue; clear_output(wait=True)
        
        for pattern_img in v:
            j = pattern_img.split('.')[0].split('_')[1]
            try:
                sem_img = Image.open(f'{dir_name}/{pattern_img}')
            except ValueError:
                print(f'{i} has no pattern found.')
                continue

            h, w = sem_img.size
            x_offset = round((bw - w) / 2)
            y_offset = round((bh - h) / 2)

            canvas = bg_img.copy()
            canvas.paste(sem_img, (y_offset, x_offset))  # Upper left corner: (0, 0)
            name = f'{i}_{j}_{dataset_name}'
            canvas.save(f'{output_path}/{name}.{extension}')
        clear_output(wait=True)

In [ ]:
dir_ = 'cropped/small_pattern'

### Train Set

In [ ]:
train_container = group_sub_pattern(train_set['SEM_No'], dir_=dir_)

In [ ]:
attach_bg(container=train_container, dir_=dir_, dataset_name='TRAIN', output_path='dataset/train/normal')

### Test Set

In [ ]:
test_normal_container = group_sub_pattern(test_set.query('Class == 1')['SEM_No'], dir_=dir_)
test_abnormal_container = group_sub_pattern(test_set.query('Class == -1')['SEM_No'], dir_=dir_)

In [ ]:
attach_bg(container=test_normal_container, dir_=dir_, dataset_name='TEST', output_path='dataset/test/normal')

In [ ]:
attach_bg(container=test_abnormal_container, dir_=dir_, dataset_name='TEST', output_path='dataset/test/abnormal')